"""classical_models.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [1]:
import sys
sys.path.append('../src/')
sys.path.append('../data/')
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from modeling_helpers import *

In [2]:
df = pd.read_pickle('../data/block4_updated.pkl')

agg_dict={
    'xtrack_dist': ['mean','std'],
    'avg_surr_speed': ['mean','std'],
    'lanes':['mean'],
    'len':['mean'],
    'speed':['mean','std'],
    'lon_acc': ['mean','std'],
    'lat_acc': ['mean','std']
}

df = df[list(agg_dict.keys()) + ['type']]
df.index = [df.index.map(lambda idx: f'{idx[0]}_{idx[1]}'),  
            df.index.get_level_values(2)]
df.index.names = ['id','road']

df = df.groupby(['id','road']).filter(lambda grp: len(grp) >= 50)
df

xtrack_dist  avg_surr_speed  lanes      len  \
id       road                                                                
4_1_1    250699362_250699984     -1.883401       10.464171    5.4   97.581   
         250699362_250699984     -1.980795       10.457843    5.4   97.581   
         250699362_250699984     -1.937041       10.452857    5.4   97.581   
         250699362_250699984     -1.893287       10.448586    5.4   97.581   
         250699362_250699984     -1.941984       10.444986    5.4   97.581   
...                                    ...             ...    ...      ...   
1_3_2116 300400248_8446047162     0.658316       26.005850    6.5  106.837   
         300400248_8446047162     0.764478       26.008150    6.5  106.837   
         300400248_8446047162     0.736857       26.010500    6.5  106.837   
         300400248_8446047162     0.783032       26.012950    6.5  106.837   
         300400248_8446047162     0.755411       26.015600    6.5  106.837   

                                 speed  lon_acc  lat_acc  type  
id       road                                                   
4_1_1    250699362_250699984   11.9046  -0.1145   0.0138  Taxi  
         250699362_250699984   11.8975  -0.1007   0.0147  Taxi  
         250699362_250699984   11.8919  -0.0918   0.0157  Taxi  
         250699362_250699984   11.8871  -0.0869   0.0167  Taxi  
         250699362_250699984   11.8831  -0.0784   0.0176  Taxi  
...                                ...      ...      ...   ...  
1_3_2116 300400248_8446047162  25.3482  -0.3011   0.3234   Car  
         300400248_8446047162  25.3217  -0.3032   0.3216   Car  
         300400248_8446047162  25.2946  -0.3012   0.3180   Car  
         300400248_8446047162  25.2667  -0.3054   0.3124   Car  
         300400248_8446047162  25.2370  -0.3135   0.3045   Car  

[2992893 rows x 8 columns]

In [3]:
# df_car = df[df.type == 'Car'].copy()
# samp = df_car.reset_index()['id'].drop_duplicates().sample(frac = 0.5)
# df_car.loc[df_car.reset_index('road', drop=True).index.isin(samp),'type']='Taxi'
# df_car

In [5]:
df_agg = downsample(df, window=50, overlap=0.3, min_speed_ratio=0.75, agg_dict=agg_dict)
model = Pipeline([('scaler', StandardScaler()), ('gbm', GradientBoostingClassifier())])
workflow(df_agg, model, StratifiedKFold(shuffle=True))

0.6360544217687075
0.608843537414966
0.6452702702702703
0.6081081081081081
0.6224489795918368


(0.6241450634307777, 0.01471261033844873)